<a href="https://colab.research.google.com/github/githubsanjana/HyperLocal-Delievery-optimizer/blob/main/Delievery_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#  Installation & Setup
# ----------------------------
# Installing essential packages for mapping, geocoding, and optimization
!pip install folium geopy ortools

# Importing required libraries
import folium  # For interactive map visualization
from geopy.distance import geodesic  # For accurate distance calculations
import random  # For generating random delivery locations
from ortools.constraint_solver import routing_enums_pb2  # Google OR-Tools
from ortools.constraint_solver import pywrapcp  # Routing solver

In [ ]:
# Data Generation
# -----------------------
# Defining central restaurant location (Connaught Place, Delhi)
restaurant = (28.6139, 77.2090)

def generate_locations(center_point, num_locations=5, radius_km=10):
    """
    Generates realistic delivery locations within specified radius
    Args:
        center_point: (lat, long) of restaurant
        num_locations: Number of delivery points
        radius_km: Maximum delivery radius in kilometers
    Returns:
        List of (latitude, longitude) tuples
    """
    return [
        geodesic(kilometers=random.uniform(0, radius_km)).destination(
            center_point, bearing=random.uniform(0, 360)
        )[:2] for _ in range(num_locations)
    ]

In [9]:
# Generating 5 delivery locations within 10km radius
delivery_points = generate_locations(restaurant)
print(f"Restaurant Location: {restaurant}")
print(f"Delivery Points: {delivery_points}")

Restaurant Location: (28.6139, 77.209)
Delivery Points: [(28.65752743484278, 77.13537591993322), (28.613340897559414, 77.21085245990496), (28.623264826340375, 77.26646495321448), (28.568873438734123, 77.20849659217954), (28.60981642012169, 77.22540999063634)]


In [10]:
# Route Optimization Engine
# --------------------------------
def create_distance_matrix(locations):
    """
    Creates distance matrix between all locations in meters
    Essential for the routing algorithm to calculate costs
    """
    return [
        [int(geodesic(i, j).meters) if i != j else 0
        for j in locations
    ] for i in locations
]

def optimize_route(locations):
    """
    Implements Google OR-Tools' advanced routing algorithm
    Solves the Traveling Salesman Problem (TSP) for optimal route
    """
    # 1. Creating distance matrix between all points
    distance_matrix = create_distance_matrix(locations)

    # 2. Initializing routing model
    manager = pywrapcp.RoutingIndexManager(len(locations), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    # 3. Defining distance calculation callback
    def distance_callback(from_index, to_index):
        return distance_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    transit_idx = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_idx)

    # 4. Configure solver parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
      # 5. Solve and extract route
    solution = routing.SolveWithParameters(search_parameters)
    index = routing.Start(0)
    route = [locations[manager.IndexToNode(index)]]

    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        route.append(locations[manager.IndexToNode(index)])

    return route

# Generate optimized delivery route
optimized_route = optimize_route([restaurant] + delivery_points)
print("Optimized Delivery Sequence:")
for i, loc in enumerate(optimized_route):
    print(f"{i}. {loc}")


Optimized Delivery Sequence:
0. (28.6139, 77.209)
1. (28.613340897559414, 77.21085245990496)
2. (28.60981642012169, 77.22540999063634)
3. (28.623264826340375, 77.26646495321448)
4. (28.568873438734123, 77.20849659217954)
5. (28.65752743484278, 77.13537591993322)
6. (28.6139, 77.209)


In [11]:
# Interactive Visualization
# --------------------------------
# Initialize map centered at restaurant
delivery_map = folium.Map(location=restaurant, zoom_start=13)

# 1. Marking  restaurant location
folium.Marker(
    location=optimized_route[0],
    tooltip="<b>Restaurant HQ</b>",
    icon=folium.Icon(color='green', icon='cutlery', prefix='fa')
).add_to(delivery_map)

# 2. Marking all delivery points
for i, point in enumerate(optimized_route[1:-1], 1):
    folium.Marker(
        location=point,
        tooltip=f"<b>Delivery #{i}</b>",
        icon=folium.Icon(color='blue', icon='shopping-cart', prefix='fa')
    ).add_to(delivery_map)

# 3. Drawing optimized route path
folium.PolyLine(
    locations=optimized_route,
    color='red',
    weight=4,
    opacity=0.8,
    tooltip="Optimized Delivery Path"
).add_to(delivery_map)

# Displaying the complete map
delivery_map